In [2]:
import time
import pycurl
import urllib
import json
import requests
import numpy as np

In [3]:
API_ENDPOINT_URL = 'http://lauzhack.sqpub.ch/'
API_KEY = '98423jhHJDmM'

class SwissQuoteStream:
    def __init__(self):
        
        self.conn = None
        self.buffer = ''
        
        
    def setup_connection(self):
        if self.conn:
            self.conn.close()
            self.buffer = ''
        self.conn = pycurl.Curl()
        self.conn.setopt(pycurl.URL, API_ENDPOINT_URL + 'prices')
        self.conn.setopt(pycurl.WRITEFUNCTION, self.callback)
    
    def start(self, callback):
        self.callback = callback
        self.setup_connection()
        backoff_network_error = 0.25
        backoff_http_error = 5
        backoff_rate_limit = 60
        while True:
            self.setup_connection()
            try:
                self.conn.perform()
            except:
                print('Network error')
                time.sleep(backoff_network_error)
                backoff_network_error = min(backoff_network_error + 1, 16)
                continue
            # HTTP Error
            sc = self.conn.getinfo(pycurl.HTTP_CODE)

            # HTTP error, use exponential back off up to 320 seconds
           # print(HTTP error %s, %s' % (sc, self.conn.errstr()))
           # print 'Waiting %s seconds' % backoff_http_error
            time.sleep(backoff_http_error)
            backoff_http_error = min(backoff_http_error * 2, 320)
            
    def buy(self, btc_amount):
        #data = str.encode('BUY 0.00000001 BTC ' + API_KEY)
        data = str.encode('BUY ' + str(btc_amount) +' BTC ' + API_KEY)
        r = requests.post(API_ENDPOINT_URL, data=data)
        return str(r.status_code) + ' ' + str(r.reason)

        
    def sell(self, btc_amount):
        #data = str.encode('SELL 0.00000001 BTC ' + API_KEY)
        data = str.encode('SELL ' + str(btc_amount) +' BTC ' + API_KEY)
        r = requests.post(API_ENDPOINT_URL, data=data)
        return str(r.status_code) + ' ' + str(r.reason)
        
    
def do_bot_things(data,time_to_sleep):
    while(True):
        
        # 1 get infos
        
        # 2 create inputs
        
        # 3 go through graph
        
        #https://stackoverflow.com/questions/37891954/keras-how-do-i-predict-after-i-trained-a-model
        #prediction = model.predict(np.array(tk.texts_to_sequences(text)))
        #print(prediction)
        
        # 4 depending on output : buy, sell or do nothing
        
        time.sleep(time_to_sleep)

In [4]:
stream = SwissQuoteStream()

In [5]:
def extract_price(data):
    return float(data.decode("utf-8").split(' ')[1][:-1])

WINDOW_MAX_SIZE = 50

window = []
epsilon = 0.01
ignore_count = 0
ignore_n = 5

normalizer_max = 5

def diff_normalizer(diff):
    if(diff > normalizer_max):
        diff_tmp = normalizer_max
    elif(diff < -normalizer_max):
        diff_tmp = -normalizer_max
    else:
        diff_tmp = diff
        
    normalized = (diff_tmp + normalizer_max) / (normalizer_max * 2)
    
    return normalized

def handle_new_price(data):
    global ignore_count

    new_price = extract_price(data)
    window.append(new_price)
    window_resized = window[-WINDOW_MAX_SIZE:]
    window_size = len(window_resized)
    
    weights = np.exp(np.linspace(-1., 0., window_size))
    weights /= weights.sum()
    
    a = np.convolve(window_resized, weights, mode='full')[:len(window_resized)]
    ema = a[window_size -1]
    
    diff = new_price - ema
    # diff > 0 -> price is above ema -> SELL SELL SELL!

    ignore_count = (ignore_count + 1 ) % ignore_n
    
    if(ignore_count == 0):
        big_enough = abs(diff) > epsilon
        
        if(not big_enough):
            print('Just HODL. Diff is: ' + str(diff))
            return
        
        END = 'http://lauzhack.sqpub.ch/teams/'

        r = requests.get(END)
        assets = r.json()[12]["assets"]
        cash = r.json()[12]["cash"]
        total = assets + cash
        
        current_percent_btc = assets / total #percentage of current bitcoin compared to total balance
        normalized_diff = 1 - diff_normalizer(diff) #percentage of bitcoin we want to have, compared to total balance
        percent_btc_diff = normalized_diff - current_percent_btc #percentage of cash to add
        amount_of_btc = percent_btc_diff * total # amount of bitcoin we want to add
        amount_in_btc = amount_of_btc / new_price #amount of equivalent bitcoin we want to add in cash
        
        print(str(total) + ' ' + str(assets) + ' ' + str(cash) + ' ||| ' + str(diff) + ' ' + str(current_percent_btc) + ' ' + str(percent_btc_diff) + ' ' + str(amount_of_btc) + ' ' + str(amount_in_btc))
        
        if(amount_in_btc < 0):
            aa = stream.sell(amount_in_btc)
            print(aa)
            #print('SELL @ ' + str(sell) + '. Diff is: ' + str(diff))
            print('Diff: ' + str(normalized_diff) + '. SELL ' + str(abs(amount_in_btc)))
        else:
            aa = stream.buy(abs(amount_in_btc))
            print(aa)
            #print('BUY @ ' + str(buy) + '. Diff is: ' + str(diff))
            print('Diff: ' + str(normalized_diff) + '. BUY ' + str(abs(amount_in_btc)))


In [ ]:
stream.start(handle_new_price)

99434.78030566842 91519.40208826419 7915.378217404222 ||| -1.1587860244212607 0.9203962819340287 -0.30451767949190256 -30279.648559469282 -8.002698050434837
200 OK
Diff: 0.6158786024421261. SELL 8.002698050434837
99382.53409207877 91467.15587467454 7915.378217404222 ||| -2.550106058512938 0.9203544336063059 -0.1653438277550121 -16432.28859877729 -4.34544191426082
200 OK
Diff: 0.7550106058512938. SELL 4.34544191426082
